In [1]:
# Amee/Shruti 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import * 
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.classification import RandomForestClassifier

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1615182176466_0007,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Amee/Shruti
ss = SparkSession.builder.config("spark.executor.memory","40g")\
                         .config("spark.driver.memory","40g")\
                         .config("spark.executor.cores",10)\
                         .config('spark.executor.instances',8)\
                         .config("spark.driver.extraClassPath", "lib/sparknlp.jar")\
                         .config("spark.driver.maxResultSize",0)\
                         .config("spark.dynamicAllocation.enabled","false")\
                         .getOrCreate()
sc = ss.sparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
for (id, rdd) in sc._jsc.getPersistentRDDs().items():
    rdd.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Amee/Shruti
train_df = ss.read.option("multiline","true")\
                  .option("header", "true").option("inferSchema", "true")\
                  .option("escape","\"").option("quote","\"")\
                  .option("mode","DROPMALFORMED").csv("s3a://msds694ww/train.csv")

train_df.cache()
train_df.show(3)

In [ ]:
#sc.install_pypi_package("sparknlp")
#import spacy
#nlp = spacy.load('en_core_web_sm')

In [ ]:
#import sparknlp
#spark=sparknlp.start()

In [ ]:
# Michelle

from io import StringIO
from html.parser import HTMLParser

import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import string

#nltk.download('stopwords')
#nltk.download('wordnet')

# Check for duplicates
There are no duplicate rows, but still remove because test set may contain

In [ ]:
# Veronica 

train_df_distinct = train_df.distinct()
train_df_distinct.cache()
train_df_distinct.show()
train_df.unpersist()

# Missing data imputation

We decided to remove rows that include missing data entirely.

In [ ]:
# Veronica  

train_df_text= train_df_distinct.filter("Title is NOT NULL OR Body is NOT NULL and Tags is NOT NULL")\
                        .withColumn("Text",concat(col("Title"),lit(" "),col("Body")))

In [ ]:
train_df_text.cache()
train_df_text.show(4)
train_df_distinct.unpersist()

In [ ]:
# def spacy_on_column(x):
    
#     doc=nlp(str(x))
#     lemmas=[token.lemma_ for token in doc]
#     a_lemmas=[lemma for lemma in lemmas if lemma.isalpha() and lemma not in stop_words]
    
#     return a_lemmas

# # spacy_on_column_udf=ss.udf.register("spacy_on_column", spacy_on_column)

In [ ]:
# train_df_spacy=train_df.withColumn("Cleaned_Text",spacy_on_column_udf(col("Text")))

In [ ]:
#train_df_spacy.show(3)

# Clean Title and Body 

In [ ]:
punctuation = string.punctuation 

In [ ]:
# Amee

# stop_words = set(stopwords.words('english')) --> Not working right now

# Temporary Stop Words workaround 
stop_words = {'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves'}

In [ ]:
# Michelle 
parser = HTMLParser()

In [ ]:
# Michelle - from: https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
remove_html_tags_udf = ss.udf.register("remove_html", remove_html_tags)


In [ ]:
# Michelle

def cleaning(df):
    
    ps = PorterStemmer()
    
    # splitting
    row = df.split(" ")
    filtered_row = []
    
    # filtering punctuation and stop words
    if len(row) > 0:
        for word in row:
            word = word.strip(r"!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'").lower()
            if word not in punctuation and word not in stop_words:
                
                # lemmatizing 
                # word = lemmatizer.lemmatize(word)
                word = ps.stem(word)
                
                                
                filtered_row.append(str(word))
        
        result=','.join(filtered_row)
    
    return result
    
cleaning_udf = ss.udf.register("cleaning", cleaning)

In [ ]:
# Michelle
HTML_strip_df = train_df_text.select(remove_html_tags_udf("Text").alias("Text"),"Title","Body","Tags")
HTML_strip_df.cache()
HTML_strip_df.show(3)

In [ ]:
train_df_text.unpersist()

## Using spacy to tokenize each row

In [ ]:
# clean_train_df=train_df.select(nlp(col("Text")).alias("clean_text"),"Text","Tags")

In [ ]:
# Michelle
cleaned_df = HTML_strip_df.select(cleaning_udf("Text").alias("Text_array"),"Title","Body",
                                  "Tags")

In [ ]:
cleaned_df.printSchema()

In [ ]:
cleaned_df.cache()
cleaned_df.show(5)
HTML_strip_df.unpersist()

## Using Bag of Word analysis

In [ ]:
# Veronica 

#cleaned_df.write.parquet("s3a://facebookmsds694/cleaned_df.parquet")

#cleaned_df.write.csv("s3a://wonderwomen/cleaned_df.csv")

In [ ]:
from pyspark.ml.feature import CountVectorizer
#val cleaned_df = spark.read.parquet("s3a://facebookmsds694/cleaned_df.parquet")
cleaned_df_array=cleaned_df.withColumn("Text_array_new",split(col("Text_array"),",").cast("array<string>"))

In [ ]:
cleaned_df_array.cache()
cleaned_df_array.show(3)
cleaned_df.unpersist()

In [ ]:
from pyspark.mllib.feature import IDF
#hashingTF = HashingTF()
#tf = hashingTF.transform(cleaned_df)
#tf.cache()
#idf = IDF(minDocFreq=2).fit(tf)
#tfidf = idfIgnore.transform(tf)
cv = CountVectorizer(inputCol="Text_array_new", outputCol="Text_Vectors", minDF=10.0,vocabSize=20000)
model=cv.fit(cleaned_df_array)
result_cv = model.transform(cleaned_df_array)

In [ ]:
#result_cv.show(4)
result_cv.write.parquet("s3a://facebookmsds694/result_cv.parquet")
cleaned_df_array.unpersist()

In [ ]:
result_cv_cache = ss.read.parquet("s3a://facebookmsds694/result_cv.parquet")

In [ ]:
result_cv_cache.show(3)

In [ ]:
num_tags=result_cv_cache.select("Tags").distinct().count()

In [ ]:
num_tags

In [ ]:
cleaned_df_array.unpersist()

In [ ]:
result_filter=result_cv.groupBy("Tags").count()
result_filter=result_filter.filter(col("Count")>300).withColumnRenamed("Tags","Tags_joined")
result_filter.cache()
result_filter.show(3)

In [ ]:
final_df=result_cv.join(result_filter,result_cv.Tags==result_filter.Tags_joined,"inner")

In [ ]:
final_df.write.parquet("s3a://facebookmsds694/final_df.parquet")
final_df.cache()
final_df.show(5)
result_filter.unpersist()
# from pyspark.ml.classification import GBTClassifier

In [5]:
final_df=ss.read.parquet("s3a://facebookmsds694/final_df.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
final_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|          Text_array|               Title|                Body|                Tags|      Text_array_new|        Text_Vectors|         Tags_joined|count|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|use,condit,given,...|how to use a whil...|<p>i am fetching ...|       php mysql sql|[use, condit, giv...|(20000,[0,1,9,22,...|       php mysql sql| 1649|
|lose,day,random,a...|I lose each day a...|<p>Ok my problem ...|         probability|[lose, day, rando...|(20000,[19,20,30,...|         probability| 3847|
|regard,painleve',...|Regarding Painlev...|<p>Painleve's fir...|    complex-analysis|[regard, painleve...|(20000,[2,66,73,9...|    complex-analysis| 2911|
|contentel,part,vi...|Why ContentElemen...|<p>I was wonderin...|      

In [7]:
c='Tags'
si = StringIndexer(inputCol=final_df.columns[3], outputCol=c + "-num")
sm = si.fit(final_df)
newdf = sm.transform(final_df).drop(c)
newdf = newdf.withColumnRenamed(c + "-num", c)
final_df.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Text_array: string, Title: string, Body: string, Tags: string, Text_array_new: array<string>, Text_Vectors: vector, Tags_joined: string, count: bigint]

In [8]:
splits = newdf.randomSplit([0.8, 0.2], seed=1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
train=splits[0].cache()
test=splits[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
train=train.select(col("Text_Vectors").alias("features"),col("Tags").alias("label"),col("count").alias("weights"))
test=test.select(col("Text_Vectors").alias("features"),col("Tags").alias("label"),col("count").alias("weights"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
sample_train=train.limit(100)
sample_test=test.limit(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
sample_test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+-------+
|            features|label|weights|
+--------------------+-----+-------+
|(20000,[12,15,19,...| 42.0|   3847|
|(20000,[5,26,42,4...|700.0|    391|
|(20000,[0,8,17,19...| 98.0|   1945|
|(20000,[8,19,32,3...|181.0|   1227|
|(20000,[0,15,24,4...|132.0|   1563|
|(20000,[1,15,26,4...|  5.0|  23751|
|(20000,[14,19,26,...| 25.0|   5107|
|(20000,[13,14,15,...| 64.0|   2636|
|(20000,[2,17,22,3...|260.0|    871|
|(20000,[5,15,35,9...|  5.0|  23751|
|(20000,[38,97,98,...|566.0|    458|
|(20000,[5,8,15,22...|385.0|    670|
|(20000,[21,33,168...| 65.0|   2616|
|(20000,[0,1,2,8,1...|212.0|   1075|
|(20000,[2,5,14,50...|951.0|    302|
|(20000,[3,6,8,27,...| 22.0|   5796|
|(20000,[2,5,8,9,1...|174.0|   1243|
|(20000,[0,2,8,12,...| 20.0|   5901|
|(20000,[1,2,3,4,6...|107.0|   1825|
|(20000,[30,40,41,...|147.0|   1431|
+--------------------+-----+-------+
only showing top 20 rows

In [ ]:
# gbt = GBTClassifier()

In [31]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1.0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
model=nb.fit(sample_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

NaiveBayes_ff757a35b673

In [34]:
Nbpredict=model.transform(sample_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
Nbpredict.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+-------+--------------------+--------------------+----------+
|            features|label|weights|       rawPrediction|         probability|prediction|
+--------------------+-----+-------+--------------------+--------------------+----------+
|(20000,[0,6,10,71...|  4.0|  24955|[-158.28443618191...|[0.00225360061706...|      10.0|
|(20000,[0,43,67,6...| 27.0|   4970|[-131.32265773759...|[0.00169011223894...|       9.0|
|(20000,[0,1,5,7,8...|239.0|    970|[-462.64919181176...|[0.00654251007738...|      10.0|
|(20000,[1,3,4,6,7...| 44.0|   3559|[-538.59288472210...|[0.05032113133079...|       7.0|
|(20000,[1,5,7,8,9...| 10.0|  15328|[-290.97639070426...|[0.70885237469591...|       0.0|
|(20000,[0,3,4,10,...| 10.0|  15328|[-3030.8157099214...|[7.06354321454360...|      27.0|
|(20000,[0,4,7,23,...|124.0|   1651|[-499.52242597826...|[7.28673651194859...|       7.0|
|(20000,[1,2,5,6,7...|240.0|    961|[-296.35166911194...|[0.00166173235045...|       7.0|
|(20000,[0

In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(Nbpredict)
#print("Model Accuracy: ", accuracy)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o584.evaluate.
: java.util.NoSuchElementException: key not found: 234.0
	at scala.collection.MapLike$class.default(MapLike.scala:228)
	at scala.collection.AbstractMap.default(Map.scala:59)
	at scala.collection.mutable.HashMap.apply(HashMap.scala:65)
	at org.apache.spark.mllib.evaluation.MulticlassMetrics.precision(MulticlassMetrics.scala:105)
	at org.apache.spark.mllib.evaluation.MulticlassMetrics.fMeasure(MulticlassMetrics.scala:124)
	at org.apache.spark.mllib.evaluation.MulticlassMetrics$$anonfun$weightedFMeasure$2.apply(MulticlassMetrics.scala:216)
	at org.apache.spark.mllib.evaluation.MulticlassMetrics$$anonfun$weightedFMeasure$2.apply(MulticlassMetrics.scala:215)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.HashMap$HashMap1.foreach(HashMap.scala:221)
	at scala.collection.immutable.HashMap$HashTri

In [ ]:
# rf = RandomForestClassifier(maxDepth=20)
# paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [20]).build()
# evaluator=MulticlassClassificationEvaluator()\
#            .setLabelCol("label")\
#            .setPredictionCol("prediction")\
#            .setMetricName("f1")

In [ ]:
#cv = CrossValidator(estimator=rf,
#                    evaluator=evaluator,
#                    numFolds=3,
#                    estimatorParamMaps=paramGrid)

In [ ]:
# model = rf.fit(train)

In [ ]:
#print("{:.{}f}".format(evaluator.evaluate(gbtpredicts), 4))

In [38]:
ss.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…